In [5]:
import collections
import contextlib
import sys
import wave
import webrtcvad
import numpy as np
import pyaudio
from time import monotonic
import malaya_speech
import numpy as np
from malaya_speech import Pipeline

In [2]:
def write_wave(path, audio, sample_rate):
    """Writes a .wav file.
    Takes path, PCM audio data, and sample rate.
    """
    with contextlib.closing(wave.open(path, 'wb')) as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(sample_rate)
        wf.writeframes(audio)

    return audio

In [3]:
def has_speech_activity(frames,number_of_frames = 25,percentage=0.85):
	# print("len_frames: ",len(frames))
	if len(frames) < number_of_frames:
		return False

	num_voiced = len([1 for frame in frames[-number_of_frames:] if vad.is_speech(frame, RATE)])
	print("num_voiced: ",num_voiced,end='\r')
	return num_voiced > percentage * number_of_frames

In [80]:
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000


VAD_ROBUSTNESS = 1
vad = webrtcvad.Vad(VAD_ROBUSTNESS)

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print(p.get_device_info_by_index(1))


def pcm2float(sig, dtype='float32'):
    """Convert PCM signal to floating point with a range from -1 to 1.
    Use dtype='float32' for single precision.
    Parameters
    ----------
    sig : array_like
        Input array, must have integral type.
    dtype : data type, optional
        Desired (floating point) data type.
    Returns
    -------
    numpy.ndarray
        Normalized floating point data.
    See Also
    --------
    float2pcm, dtype
    """
    sig = np.asarray(sig)
    if sig.dtype.kind not in 'iu':
        raise TypeError("'sig' must be an array of integers")
    dtype = np.dtype(dtype)
    if dtype.kind != 'f':
        raise TypeError("'dtype' must be a floating point type")

    i = np.iinfo(sig.dtype)
    abs_max = 2 ** (i.bits - 1)
    offset = i.min + abs_max
    return (sig.astype(dtype) - offset) / abs_max



MIN_CONSECUTIVE_FRAMES = 25
MAX_WAIT = 2


for i in range(1):
	consecutive_frames_reached = 0
	frames = []
# 	int_frames = []
	last_frames = []
	has_spoken = False

	silent_time = None


	while True:
        
	    raw = stream.read(CHUNK, exception_on_overflow=False)
#         with open('vad_test_file.wav', 'rb') as audio_file:
#             content = audio_file.read(BYTES_PER_SECOND)
        
	    _last_frame = np.fromstring(raw, dtype=np.int16)
	    
	    pcm = _last_frame.tostring()

	    if has_speech_activity(last_frames):
	        silent_time = None
	        print("Consecutive frames detected!!!")
	        has_spoken = True
	    else:
	        if silent_time is None:
	            silent_time = monotonic()
	        elif monotonic() - silent_time > MAX_WAIT and has_spoken:
	            print("Voice command ended!!!")
	            y = write_wave(f"test_{i}.wav",b''.join([f for f in frames]),RATE)
	            break
	    last_frames.append(_last_frame[0:480].tostring())
	    frames.append(pcm)
        
int_frames = pcm2float(np.frombuffer(raw, dtype=np.int16), dtype='float32')

y_int = malaya_speech.astype.float_to_int(int_frames)
# vad = malaya_speech.vad.webrtc(minimum_amplitude=int(np.quantile(np.abs(y_int), 0.2)))
malaya_speech.vad.available_model()
model_factor1 = malaya_speech.vad.deep_model(model='marblenet-factor1')
frames_int = list(malaya_speech.utils.generator.frames(y_int, 30, RATE))


frames = list(malaya_speech.utils.generator.frames(int_frames, 30, RATE))

frames_deep_factor1 = [(frame, model_factor1(frame)) for frame in frames]


{'index': 1, 'structVersion': 2, 'name': 'Microphone Array (Realtek(R) Au', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}


C:\Users\developer\Desktop\uluc_nemo\uluc_nemo\lib\site-packages\ipykernel_launcher.py:72: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
C:\Users\developer\Desktop\uluc_nemo\uluc_nemo\lib\site-packages\ipykernel_launcher.py:74: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
C:\Users\developer\Desktop\uluc_nemo\uluc_nemo\lib\site-packages\ipykernel_launcher.py:87: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.


KeyboardInterrupt: 

In [58]:
int_frames

array([ 3.3569336e-04,  2.4414062e-04, -7.6293945e-04, ...,
        5.1879883e-04,  3.6621094e-04, -3.0517578e-05], dtype=float32)

In [5]:
y_int = malaya_speech.astype.float_to_int(last_frames)
vad = malaya_speech.vad.webrtc(minimum_amplitude=int(np.quantile(np.abs(y_int), 0.2)))
malaya_speech.vad.available_model()
model_factor1 = malaya_speech.vad.deep_model(model='marblenet-factor1')
frames_int = list(malaya_speech.utils.generator.frames(y_int, 30, RATE))


frames = list(malaya_speech.utils.generator.frames(last_frames, 30, RATE))

frames_deep_factor1 = [(frame, model_factor1(frame)) for frame in frames]

p = Pipeline()
pipeline = (
	p.batching(5)
		.foreach_map(model_factor1.predict)
		.flatten()
)
p.visualize()

result = p.emit(frames)
result.keys()

frames_deep_factor1_batch = [(frame, result['flatten'][no]) for no, frame in enumerate(frames)]
len(frames_deep_factor1) == len(frames_deep_factor1_batch)

malaya_speech.extra.visualization.visualize_vad(y, frames_deep_factor1, RATE)


UFuncTypeError: ufunc 'absolute' did not contain a loop with signature matching types dtype('S960') -> dtype('S960')

In [6]:
y, sr = malaya_speech.load('vad_test_file.wav')

y.shape

(399701,)

In [7]:
import soundfile as sf
from scipy.io.wavfile import read
from scipy import interpolate

def read_audio(data, old_samplerate, sample_rate=22050):
    if len(data.shape) == 2:
        data = data[:, 0]

    if old_samplerate != sample_rate and sample_rate is not None:
        data = resample(data, old_samplerate, sample_rate)
    else:
        sample_rate = old_samplerate

    return data, sample_rate

scale = True

data, old_samplerate = sf.read('vad_test_file.wav')
y, sr = read_audio(data, old_samplerate, sr)
if scale == True:
    y = y / (np.max(np.abs(y)) + 1e-9)

NameError: name 'resample' is not defined

In [36]:
y

array([ 0.        ,  0.        ,  0.        , ...,  0.0015564 ,
       -0.00268555, -0.00015259])

In [10]:
y_int = malaya_speech.astype.float_to_int(y)
# vad = malaya_speech.vad.webrtc(minimum_amplitude=int(np.quantile(np.abs(y_int), 0.2)))
malaya_speech.vad.available_model()
model_factor1 = malaya_speech.vad.deep_model(model='marblenet-factor1')
frames_int = list(malaya_speech.utils.generator.frames(y_int, 30, RATE))


frames = list(malaya_speech.utils.generator.frames(y, 30, RATE))

frames_deep_factor1 = [(frame, model_factor1(frame)) for frame in frames]


In [12]:
y[0]

0.0